In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification 
from sklearn.model_selection import train_test_split 
import tensorflow as tf 

import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# split x and y
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')

In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
import string 
from nltk.corpus import stopwords 
def text_process(mess):
    """ Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords 
    3. Returns a list of the cleaned text """ 
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'] # Check characters to see if they are in punctuation 
    nopunc = [char for char in mess if char not in string.punctuation] # Join the characters again to form the string.
    nopunc = ''.join(nopunc) # Now just remove any stopwords 
    nopunc = nopunc.lower()
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])


def split_x_y(df):
    df['text'] = df['v2'].apply(text_process)
    
    label_dict = {'ham': 0, 'spam' :1}
    df['labels'] = df['v1'].map(label_dict)
    
    train_texts = df['text'].values
    train_labels = df['labels'].values
    
    return train_texts, train_labels


def data_preprocess(df):
    
    train_texts, train_labels = split_x_y(df)
    train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.05)
    
    train_encodings = tokenizer(list(train_texts), truncation=True, padding=True) 
    val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
    
    train_dataset = tf.data.Dataset.from_tensor_slices(( dict(train_encodings), train_labels ))
    val_dataset = tf.data.Dataset.from_tensor_slices(( dict(val_encodings), val_labels ))
    return train_dataset, val_dataset

In [6]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
train_dataset, val_dataset = data_preprocess(df)

2022-06-18 06:16:52.333837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 06:16:52.334980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 06:16:52.335663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 06:16:52.337655: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss) # can also use any keras loss fn 

model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

2022-06-18 06:17:07.438723: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

Epoch 1/3


/opt/conda/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)
2022-06-18 06:17:18.207254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


331/331 [==============================] - 50s 118ms/step - loss: 0.0787
Epoch 2/3
331/331 [==============================] - 39s 117ms/step - loss: 0.0295
Epoch 3/3
331/331 [==============================] - 39s 117ms/step - loss: 0.0151


In [9]:
model.save('./my_model')

In [10]:
os.listdir('./my_model/')

['keras_metadata.pb', 'saved_model.pb', 'variables', 'assets']

In [11]:
# from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
# training_args = TFTrainingArguments( output_dir='./results', # output directory
#                                     num_train_epochs=3, # total number of training epochs 
#                                     per_device_train_batch_size=16, # batch size per device during training 
#                                     per_device_eval_batch_size=64, # batch size for evaluation
#                                     warmup_steps=500, # number of warmup steps for learning rate scheduler
#                                     weight_decay=0.01, # strength of weight decay 
#                                     logging_dir='./logs', # directory for storing logs
#                                     logging_steps=10, ) 

# with training_args.strategy.scope(): 
#     model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# trainer = TFTrainer( model=model, # the instantiated 🤗 Transformers model to be trained 
#                     args=training_args, # training arguments, defined above 
#                     train_dataset=train_dataset, # training dataset
#                     eval_dataset=val_dataset # evaluation dataset 
#                    )

# trainer.train()

https://huggingface.co/transformers/v3.2.0/custom_datasets.html